# ST-GAN

In [1]:
import torch.utils.data as data

from PIL import Image
import os
import os.path
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.nn import init
#import transforms.ISTD_transforms as transforms

from torch.autograd import Variable
import torchvision.utils as vutils
import matplotlib.pyplot as plt

In [2]:
class inconv(nn.Module): #Extracting feature with kernel (4x4), stride = 2, padding = 1
    def __init__(self, in_channel, out_channel):
        super(inconv, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=4, stride=2, padding=1, bias=False)

    def forward(self, x):
        x = self.conv(x)
        return x

class lrl_conv_bn(nn.Module): #Extracting feature with kernel (4x4), stride = 2, padding = 1 but leaky_relu is before and batchnorm2d is after
    '''Leaky ReLU -> Conv -> BN'''
    def __init__(self, in_ch, out_ch):
        super(lrl_conv_bn, self).__init__()
        self.conv_bn = nn.Sequential(
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        x = self.conv_bn(x)
        return x

class lrl_conv_bn_triple(nn.Module): #Triple layer
    '''Leaky ReLU -> Conv -> BN'''
    def __init__(self, in_ch, out_ch):
        super(lrl_conv_bn_triple, self).__init__()
        self.conv_bn = nn.Sequential(
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        x = self.conv_bn(x)
        return x

class lrl_conv(nn.Module): #Without batch normalization
    '''Leaky ReLU -> Conv'''
    def __init__(self, in_ch, out_ch):
        super(lrl_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class rl_convT_bn(nn.Module): #Relu but not leaky relu
    '''ReLU -> ConvT -> BN'''
    def __init__(self, in_ch, out_ch):
        super(rl_convT_bn, self).__init__()
        self.conv = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class rl_convT_bn_triple(nn.Module): #Triple relu
    '''ReLU -> ConvT -> BN'''
    def __init__(self, in_ch, out_ch):
        super(rl_convT_bn_triple, self).__init__()
        self.conv = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(out_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(out_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class rl_convT(nn.Module): #Relu without batch
    '''ReLU -> ConvT'''
    def __init__(self, in_ch, out_ch):
        super(rl_convT, self).__init__()
        self.conv = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [3]:
class generator(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(generator, self).__init__()
        self.inc = inconv(in_ch, 64)
        self.conv_1 = lrl_conv_bn(64, 128)
        self.conv_2 = lrl_conv_bn(128, 256)
        self.conv_3 = lrl_conv_bn(256, 512)
        self.conv_4 = lrl_conv_bn_triple(512, 512)
        self.conv_5 = lrl_conv(512, 512)
        self.conv_T6 = rl_convT_bn(512, 512)
        self.conv_T7 = rl_convT_bn_triple(1024,512)
        self.conv_T8 = rl_convT_bn(1024, 256)
        self.conv_T9 = rl_convT_bn(512, 128)
        self.conv_T10 = rl_convT_bn(256, 64)
        self.conv_T11 = rl_convT(128, out_ch)

    def forward(self, input):
        cv0 = self.inc(input)
        cv1 = self.conv_1(cv0)
        cv2 = self.conv_2(cv1)
        cv3 = self.conv_3(cv2)
        cv4 = self.conv_4(cv3)
        cv5 = self.conv_5(cv4)
        cvT6 = self.conv_T6(cv5)
        input7 = torch.cat([cvT6, cv4], dim=1)
        cvT7 = self.conv_T7(input7)
        input8 = torch.cat([cvT7, cv3], dim=1)
        cvT8 = self.conv_T8(input8)
        input9 = torch.cat([cvT8, cv2], dim=1)
        cvT9 = self.conv_T9(input9)
        input10 = torch.cat([cvT9, cv1], dim=1)
        cvT10 = self.conv_T10(input10)
        input11 = torch.cat([cvT10, cv0], dim=1)
        cvT11 = self.conv_T11(input11)
        out = torch.tanh(cvT11)
        return out

class discriminator(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(discriminator, self).__init__()
        self.inc = inconv(in_ch, 64)
        self.conv_1 = lrl_conv_bn(64, 128)
        self.conv_2 = lrl_conv_bn(128, 256)
        self.conv_3 = lrl_conv_bn(256, 512)
        self.conv_4 = lrl_conv(512, out_ch)

    def forward(self, input):
        cv0 = self.inc(input)
        cv1 = self.conv_1(cv0)
        cv2 = self.conv_2(cv1)
        cv3 = self.conv_3(cv2)
        cv4 = self.conv_4(cv3)
        out = torch.sigmoid(cv4)
        return out

In [4]:
G1_trained = generator(3, 1)
G2_trained = generator(4, 3)
D1_trained = discriminator(4, 1)
D2_trained = discriminator(7, 1)

root_path = os.getcwd()
model_path = os.path.dirname(root_path) + "\\checkpoints_STGAN"

In [5]:
num_epoch = 49
G1_trained.load_state_dict(torch.load(model_path+"\\G1_" + str(num_epoch) + ".pth"))
G1_trained.eval()
G2_trained.load_state_dict(torch.load(model_path+"\\G2_" + str(num_epoch) + ".pth"))
G2_trained.eval()
D1_trained.load_state_dict(torch.load(model_path+"\\D1_" + str(num_epoch) + ".pth"))
D1_trained.eval()
D2_trained.load_state_dict(torch.load(model_path+"\\D2_" + str(num_epoch) + ".pth"))
D2_trained.eval()

discriminator(
  (inc): inconv(
    (conv): Conv2d(7, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv_1): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv_2): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv_3): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [6]:
# Send to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda_tensor = torch.cuda.FloatTensor if device == torch.device('cuda:0') else torch.FloatTensor
G1_trained.to(device)
G2_trained.to(device)
D1_trained.to(device)
D2_trained.to(device)

discriminator(
  (inc): inconv(
    (conv): Conv2d(7, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv_1): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv_2): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv_3): lrl_conv_bn(
    (conv_bn): Sequential(
      (0): LeakyReLU(negative_slope=0.2, inplace=True)
      (1): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runnin

# SP-M NET

In [7]:
from options.train_options import TrainOptions
from options.test_options import TestOptions
from data import CustomDatasetDataLoader
from models import create_model

from PIL import Image

import torch
import torch.nn.functional as F
import numpy as np
import util.util as util
import matplotlib.pyplot as plt
import cv2

In [8]:
opt = TestOptions().parse()

----------------- Options ---------------
               batch_size: 2                             
          checkpoints_dir: C:/Users/m1101/Downloads/Shadow_Removal/SID/_Git_SID/checkpoints_PAMI/
                 dataroot: C:/Users/m1101/Downloads/Shadow_Removal/SID/_Git_SID/data_processing/dataset/NTUST_TU/train/
             dataset_mode: shadowparam                   
                    epoch: latest                        
                 fineSize: 256                           
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
               lambda_GAN: 0.0                           
                lambda_L1: 100                           
            lambda_smooth: 0.0                           
                 loadSize: 2

In [9]:
data_loader = CustomDatasetDataLoader(opt) #createDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)

dataset [ShadowParamDataset] was created


In [10]:
test_model = create_model(opt)
test_model.setup(opt)
test_model.load_networks(50)

initialize network with xavier
model [Shadow Image Decomposition model ICCV19] was created
<bound method SIDModel.name of <models.model_SID.SIDModel object at 0x00000294FBB92430>>
LOADING <bound method SIDModel.name of <models.model_SID.SIDModel object at 0x00000294FBB92430>>
loading the model from C:/Users/m1101/Downloads/Shadow_Removal/SID/_Git_SID/checkpoints_PAMI/SID_GRESNEXT_shadowparam\latest_net_G.pth
loading the model from C:/Users/m1101/Downloads/Shadow_Removal/SID/_Git_SID/checkpoints_PAMI/SID_GRESNEXT_shadowparam\latest_net_M.pth
---------- Networks initialized -------------
DataParallel(
  (module): ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2

loading the model from C:/Users/m1101/Downloads/Shadow_Removal/SID/_Git_SID/checkpoints_PAMI/SID_GRESNEXT_shadowparam\50_net_M.pth


In [11]:
# def run(input_img, shadow_mask):
#     inputG = torch.cat([input_img,shadow_mask],1)
#     shadow_mask = (shadow_mask>0.9)*2-1
#     shadow_mask_3d= (shadow_mask>0).expand(input_img.shape)   

#     inputG = F.interpolate(inputG,size=(256,256))
#     shadow_param_pred = test_model.netG(inputG)
#     w = input_img.shape[2]
#     h = input_img.shape[3]
#     n = input_img.shape[0]
#     m = input_img.shape[1]
#     shadow_param_pred = shadow_param_pred.view([n,6,-1])
#     shadow_param_pred = torch.mean(shadow_param_pred,dim=2)
#     shadow_param_pred[:,[1,3,5]] = (shadow_param_pred[:,[1,3,5]]*2)+3 

#     lit = input_img.clone()/2+0.5
#     add = shadow_param_pred[:,[0,2,4]]
#     mul = shadow_param_pred[:,[1,3,5]]
#     #mul = (mul +2) * 5/3          
#     n = shadow_param_pred.shape[0]
#     add = add.view(n,3,1,1).expand((n,3,w,h))
#     mul = mul.view(n,3,1,1).expand((n,3,w,h))
#     lit = lit*mul + add
#     out = (input_img/2+0.5)*~(shadow_mask_3d) + lit*shadow_mask_3d
#     out = out*2-1

#     inputM = torch.cat([input_img,lit,shadow_mask],1)
#     alpha_pred = test_model.netM(inputM)
#     alpha_pred = (alpha_pred +1) /2        
#     #alpha_pred_3d=  alpha_pred.repeat(1,3,1,1)

#     final = (input_img/2+0.5)*(1-alpha_pred) + lit*alpha_pred
#     final = final*2-1

#     RES = dict()
#     RES['final']= util.tensor2im(final,scale =0)
#     RES['phase1'] = util.tensor2im(out,scale =0)
#     RES['param']= shadow_param_pred.detach().cpu() 
#     RES['matte'] = util.tensor2im(alpha_pred.detach().cpu()/2,scale =0)
    
#     return RES

# ASSESS AND SAVE IMAGE

In [12]:
import sys
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.3+
    count = len(it)
    def show(j, batch_size):
        n = batch_size*j
        x = int(size*n/count)
        print("{}[{}{}] {}/{}".format(prefix, "#"*x, "."*(size-x), n, count), 
                end='\r', file=out, flush=True)
    show(0, 1)
    for i, item in enumerate(it):
        yield item
        batch_size = len(list(item.values())[0])
        show(i+1, batch_size)
    print("\n", flush=True, file=out)

In [13]:
# import cv2

# def calculate_psnr(img1, img2):
#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)
#     mse = np.mean((img1 - img2)**2)
#     if mse == 0:
#         return float('inf')
#     return 20 * math.log10(255.0 / math.sqrt(mse))

# def ssim(img1, img2):
#     C1 = (0.01 * 255)**2
#     C2 = (0.03 * 255)**2

#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)
#     kernel = cv2.getGaussianKernel(11, 1.5)
#     window = np.outer(kernel, kernel.transpose())

#     mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
#     mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
#     mu1_sq = mu1**2
#     mu2_sq = mu2**2
#     mu1_mu2 = mu1 * mu2
#     sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
#     sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
#     sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

#     ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
#                                                             (sigma1_sq + sigma2_sq + C2))
#     return ssim_map.mean()


# def calculate_ssim(img1, img2):
#     '''calculate SSIM
#     the same outputs as MATLAB's
#     img1, img2: [0, 255]
#     '''
#     if not img1.shape == img2.shape:
#         raise ValueError('Input images must have the same dimensions.')
#     if img1.ndim == 2:
#         return ssim(img1, img2)
#     elif img1.ndim == 3:
#         if img1.shape[2] == 3:
#             ssims = []
#             for i in range(3):
#                 ssims.append(ssim(img1, img2))
#             return np.array(ssims).mean()
#         elif img1.shape[2] == 1:
#             return ssim(np.squeeze(img1), np.squeeze(img2))
#     else:
#         raise ValueError('Wrong input image dimensions.')

## Assessing and Saving Images at once

In [14]:
from pathlib import Path
Path(os.getcwd() + f"\\result_set\\original").mkdir(parents=True, exist_ok=True)
Path(os.getcwd() + f"\\result_set\\stgan").mkdir(parents=True, exist_ok=True)
Path(os.getcwd() + f"\\result_set\\sid").mkdir(parents=True, exist_ok=True)
Path(os.getcwd() + f"\\result_set\\groudtruth").mkdir(parents=True, exist_ok=True)

In [15]:
# import math

# my_psnr = 0
# my_psnr_A = 0
# my_psnr_B = 0

# my_ssim = 0
# my_ssim_A = 0
# my_ssim_B = 0

# epoch_index = -1
# for data in progressbar(dataset):
#     list_A = Variable(data['A'].type(cuda_tensor), 0)
#     list_C = Variable(data['C'].type(cuda_tensor), 0)
#     list_imname = data['imname']
    
#     list_A = torch.reshape(list_A,(-1,3,256,256))
#     list_C = torch.reshape(list_C,(-1,3,256,256))
#     input_img = list_A 
#     shadow_mask = G1_trained(input_img) 
#     img_fake_B = (shadow_mask[0] + 1.0)/2.0
#     blur_ = cv2.GaussianBlur(img_fake_B[0].cpu().detach().numpy(), (0,0), 4.0, 4.0)
#     blur = torch.from_numpy(blur_).type(cuda_tensor)[None, None, :, :]
#     blur = (blur - 0.5)*2.0
    
#     fake_C_ST_GAN = G2_trained(torch.cat((input_img, shadow_mask), 1))
#     RES = run(input_img, blur)
    
#     epoch_index += 1
#     for i in range(len(list_A)):
#         img_real_A = list_A[i].permute(1, 2, 0)
#         img_real_C = list_C[i].permute(1, 2, 0)
#         img_fake_C = fake_C_ST_GAN[i].data.permute(1, 2, 0)

#         #Scale to 255
#         img_real_A = ((img_real_A + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8)
#         img_real_C = ((img_real_C + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8) 
#         img_fake_C = ((img_fake_C + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8) 
#         img_fake_C_SID = RES['final']
        
#         data_name = os.getcwd() + f"\\result_set\\original\\{list_imname[i]}"
#         Image.fromarray(img_real_A).convert('RGB').resize((224, 224)).save(data_name)
        
#         data_name = os.getcwd() + f"\\result_set\\stgan\\{list_imname[i]}"              
#         Image.fromarray(img_fake_C).convert('RGB').resize((224, 224)).save(data_name)
        
#         data_name = os.getcwd() + f"\\result_set\\sid\\{list_imname[i]}"              
#         Image.fromarray(img_fake_C_SID).convert('RGB').resize((224, 224)).save(data_name)
        
#         data_name = os.getcwd() + f"\\result_set\\groudtruth\\{list_imname[i]}"              
#         Image.fromarray(img_real_C).convert('RGB').resize((224, 224)).save(data_name)

#         my_psnr += calculate_psnr(img_real_C, img_real_A)
#         my_ssim += calculate_ssim(img_real_C, img_real_A)

#         my_psnr_A += calculate_psnr(img_real_C, img_fake_C)
#         my_ssim_A += calculate_ssim(img_real_C, img_fake_C)
        
#         my_psnr_B += calculate_psnr(img_real_C, img_fake_C_SID)
#         my_ssim_B += calculate_ssim(img_real_C, img_fake_C_SID)
        
# #         if epoch_index == 20:
# #             break
    
# length = len(list_A)*len(dataset)
# print("Amount of data: ", length)
# print("PSNR 0: ", my_psnr/length, "SSIM 0: ", my_ssim/length)
# print("PSNR A: ", my_psnr_A/length, "SSIM A: ", my_ssim_A/length)
# print("PSNR B: ", my_psnr_B/length, "SSIM B: ", my_ssim_B/length)

In [ ]:
from models.loss_function import calculate_ssim, calculate_psnr

my_psnr = 0
my_psnr_A = 0
my_psnr_B = 0

my_ssim = 0
my_ssim_A = 0
my_ssim_B = 0

epoch_index = -1
for data in progressbar(dataset):
    list_A = Variable(data['A'].type(cuda_tensor), 0)
    list_C = Variable(data['C'].type(cuda_tensor), 0)
    list_imname = data['imname']
    
    list_A = torch.reshape(list_A,(-1,3,256,256))
    list_C = torch.reshape(list_C,(-1,3,256,256))
    input_img = list_A 
    shadow_mask = G1_trained(input_img) 
    
    # blur shadow image
    img_fake_B = (shadow_mask + 1.0)/2.0
#     for i_blur in range(len(img_fake_B)):
#         blur_ = cv2.GaussianBlur(img_fake_B[i_blur].cpu().detach().numpy(), (0,0), 4.0, 4.0)
#         img_fake_B[i_blur] = torch.from_numpy(blur_).type(cuda_tensor)[None, None, :, :]
    img_fake_B = (img_fake_B - 0.5)*2.0
    
    fake_C_ST_GAN = G2_trained(torch.cat((input_img, img_fake_B), 1))
    RES = test_model.get_prediction(input_img, img_fake_B)
    
    epoch_index += 1
    for i in range(len(list_A)):
        img_real_A = list_A[i].permute(1, 2, 0)
        img_real_C = list_C[i].permute(1, 2, 0)
        img_fake_C = fake_C_ST_GAN[i].data.permute(1, 2, 0)
        img_fake_C_SID = RES['final'][i].data.permute(1, 2, 0)
        
        my_psnr += calculate_psnr(img_real_C, img_real_A)
        my_ssim += calculate_ssim(img_real_C, img_real_A)

        my_psnr_A += calculate_psnr(img_real_C, img_fake_C)
        my_ssim_A += calculate_ssim(img_real_C, img_fake_C)
        
        my_psnr_B += calculate_psnr(img_real_C, img_fake_C_SID)
        my_ssim_B += calculate_ssim(img_real_C, img_fake_C_SID)

        #Scale to 255
        img_real_A = ((img_real_A + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8)
        img_real_C = ((img_real_C + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8) 
        img_fake_C = ((img_fake_C + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8) 
        img_fake_C_SID = ((img_fake_C_SID + 1.0)*255.0/2.0).cpu().numpy().astype(np.uint8) 

        
        data_name = os.getcwd() + f"\\result_set\\original\\{list_imname[i]}"
        Image.fromarray(img_real_A).convert('RGB').resize((224, 224)).save(data_name)
        
        data_name = os.getcwd() + f"\\result_set\\stgan\\{list_imname[i]}"              
        Image.fromarray(img_fake_C).convert('RGB').resize((224, 224)).save(data_name)
        
        data_name = os.getcwd() + f"\\result_set\\sid\\{list_imname[i]}"              
        Image.fromarray(img_fake_C_SID).convert('RGB').resize((224, 224)).save(data_name)
        
        data_name = os.getcwd() + f"\\result_set\\groudtruth\\{list_imname[i]}"              
        Image.fromarray(img_real_C).convert('RGB').resize((224, 224)).save(data_name)
        
#     if epoch_index == 20:
#         break
    
length = len(dataset)
print("Amount of data: ", length)
print("PSNR 0: ", my_psnr/length, "SSIM 0: ", my_ssim/length)
print("PSNR A: ", my_psnr_A/length, "SSIM A: ", my_ssim_A/length)
print("PSNR B: ", my_psnr_B/length, "SSIM B: ", my_ssim_B/length)